In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_simulation_bayesimp as file

In [2]:
# Args setup
ntrial = 50
n = 500
n_int = 100
niter = 1000
front_door = True
minimise = False
add_base_kernel_BO = True

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32951 instead
  warnings.warn(


In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                              seed,
                              n,n_int,
                              niter = niter,
                              front_door = front_door,
                              minimise = minimise,
                              add_base_kernel_BO = add_base_kernel_BO
                             )
        futures += [f]

In [5]:
futures

[<Future: pending, key: main-cd79934cfbfff32fbaecb605ebc707e7>,
 <Future: pending, key: main-aa70d890d7fbf5f753b749413504d27c>,
 <Future: pending, key: main-7bd7deb615337395b34e20f743c686cb>,
 <Future: pending, key: main-e876430ed5f48e55e10dcb98ca679b55>,
 <Future: pending, key: main-1b151d39b85ee92001b9be5d25740bef>,
 <Future: pending, key: main-65b6f62ae368e6299d0ae3a3f8c7e543>,
 <Future: pending, key: main-a88180b233eb595937e4621c8b9a0173>,
 <Future: pending, key: main-e56677e352eb38d1a640fcffb87a3878>,
 <Future: pending, key: main-1e8bcbc27b6fb7a011bd8ca22066ac0c>,
 <Future: pending, key: main-3650d2dd3d9f2f4df76aaea760adbb29>,
 <Future: pending, key: main-6da3b8f97d2105b70beeaa02a5be5177>,
 <Future: pending, key: main-7583d9f2485f589e278bc136f3a7d918>,
 <Future: pending, key: main-4e5fb8d20e0bd296c66ce05e3f757e24>,
 <Future: pending, key: main-d648863ac624dcdd4eb81ed868228b0f>,
 <Future: pending, key: main-440522a6474655fc3d9901bde7f9112f>,
 <Future: pending, key: main-ace2fc00462

In [ ]:
# Getting results
results = client.gather(futures)

In [ ]:
# Closing client
client.close()
cluster.close()

In [ ]:
# Saving results
torch.save(obj = results,
           f = "simulation_bayesimp_ntrial={0}_n={1}_frontdoor={2}_minimise={3}_addbase={4}.pt".format(ntrial,
                                                                                           n,
                                                                                           front_door,
                                                                                           minimise,
                                                                                           add_base_kernel_BO)
          )